In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('Datasets/backup.parquet')

In [2]:
np.array(df.Photo_Url.unique().shape) - np.array(df.Photo_Url.shape)

array([-13531])

In [3]:
df.dtypes

Type                               object
Org                              category
Description                        object
Ticket_Id                          object
Coords                             object
Photo_Url                          object
After_Photo                        object
Address                            object
Timestamp             datetime64[ns, UTC]
Problem_Type_Abdul                 object
Star                              float64
Count_Reopen                        int64
Note                               object
State                              object
Last_Activity         datetime64[ns, UTC]
dtype: object

In [4]:
import ray

ray.init()

2023-03-18 14:10:33,262	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.9
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [20]:
# check if all filename from url is unique
df.Photo_Url.apply(lambda x: x.split('/')[-1]).unique().shape[0] - df.Photo_Url.apply(lambda x: x.split('/')[-1]).shape[0]

-13531

In [8]:
urls = df.Photo_Url.dropna().unique().tolist()
urls += df.After_Photo.dropna().unique().tolist()
urls, len(urls)

(['https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/c95301eddca89cb2b6a5c5bfd991c9f93c4a35e3.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/aaa511700204b9092c16a172caed269a22c9751a.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/8b1bb8792ea6c946cf993d0a1319dbdb199e060e.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/1677599884553.jpeg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/08c0b8e603be272f1591e8338072ee173caa55b1.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/92909ed78fc9ac4e9c1139d919f70ee8acf3c2ce.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/bcf4845f2779f799752ddcfd7bf4d30c671e73e0.jpg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/1677599333061.jpeg',
  'https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/1677599130

In [11]:
# download images to local folder
import requests
import os
from pathlib import Path
import time
from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool
from ulid import ULID
from urllib.parse import unquote, urlparse

output_dir = Path('Datasets/bkkchangelog_images2')
output_dir.mkdir(exist_ok=True)

def get_extension(url):
    return os.path.splitext(unquote(urlparse(url).path))[1]

@ray.remote
def download_url(url):
    try:
        r = requests.get(url)
        ulid = ULID()
        ext = get_extension(url)
        filename = output_dir / f'{str(ulid)}{ext}'
        with filename.open('wb') as f:
            f.write(r.content)
        return url, filename
    except Exception as e:
        return url, None

# test 1 download
# download_url(df.Photo_Url[0], Path('Datasets/images/0.jpg'))

# test get_extension
# get_extension('https://storage.googleapis.com/traffy_public_bucket/attachment/2023-02/c95301eddca89cb2b6a5c5bfd991c9f93c4a35e3.jpg')

# download all 100 images and get avg size
# first_100 = urls[:100]
# first_100 = [download_url.remote(url) for url in first_100]
# first_100 = ray.get(first_100)




In [14]:
# get size of output_dir
import os
from pathlib import Path
output_dir = Path('Datasets/bkkchangelog_images')
size4100_MB = sum(os.path.getsize(f) for f in output_dir.glob('**/*') if f.is_file()) / 1024 / 1024
size4100_MB


24.69024085998535

In [15]:
# project to entire images
size4100_MB * len(urls) / 100

99572.77855941773

In [16]:
# download all images
%timeit
all_images = [download_url.remote(url) for url in urls]
all_images = ray.get(all_images)

In [ ]:
df_image_dl = pd.DataFrame(all_images)
df_image_dl.columns = ['Url', 'Filename']
df_image_dl.to_parquet('Datasets/image_dl.parquet')

In [12]:
# check exif
from PIL import Image
from PIL.ExifTags import TAGS

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    raw_exif = image._getexif()
    if raw_exif is None:
        return {}
    exif = {}
    for tag, value in raw_exif.items():
        decoded = TAGS.get(tag, tag)
        exif[decoded] = value
    return exif

# test
get_exif('Datasets/images/0.jpg')

{'ExifOffset': 38,
 'Orientation': 1,
 'ColorSpace': 65535,
 'ExifImageWidth': 1774,
 'ExifImageHeight': 2364}